In [1]:
from utils import * 
import pandas as pd
import altair as alt

In [2]:
# Load in our data
# NFL 2013 QB data 
# Surprise = Travarious Jackson --> round 2 pick 64
# Bust = Brandon Weeden --> 22 overall pick

import chardet
with open(r'QBStats_2013.csv', 'rb') as f:
    result = chardet.detect(f.read())
    df = pd.read_csv(r'QBStats_2013.csv', encoding=result['encoding'])
    
df = df.head(100)
df

,qb,att,cmp,yds,ypa,td,int,lg,sack,loss,rate,game_points,home_away,year
0,Joe FlaccoJ. Flacco,62,34,362,5.8,2,2,34,4,27,69.4,27,away,2013
1,Peyton ManningP. Manning,42,27,462,11.0,7,0,78t,3,17,141.1,49,home,2013
2,Jake LockerJ. Locker,20,11,125,6.2,0,0,25,1,8,74.0,16,away,2013
3,Ben RoethlisbergerB. Roethlisberger,33,21,191,5.8,1,1,22,5,28,76.7,9,home,2013
4,Ryan TannehillR. Tannehill,38,24,272,7.2,1,1,34t,4,17,82.3,23,away,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Russell WilsonR. Wilson,21,14,202,9.6,4,1,30,2,8,117.5,45,home,2013
96,Tarvaris JacksonT. Jackson,8,7,129,16.1,1,0,35t,0,0,158.3,45,home,2013
97,Andrew LuckA. Luck,26,17,159,6.1,0,0,25,1,7,82.1,27,away,2013
98,Colin KaepernickC. Kaepernick,27,13,150,5.6,0,1,30,3,11,49.9,7,home,2013


In [6]:
# qb_grouped = df.groupby('qb')

# print (qb_grouped)

# was going to groupby quarterback but decided not to for now!

## Visualization 1. 
### Look at all QB's in dataset
### Compate their passing attempts (x) to completions (y)
### When hover over a circle you can see the QB name and their passer rating for that game

In [7]:
# Add interaction to figure out which player is which
alt.Chart(df).mark_point().encode(
    x = "att",
    y = "cmp",
    tooltip=["qb", "rate"]
).interactive()

alt.Chart(...)

## You can see that Travarious Jackson has a high average rating
## Whereas Brandon Weeden has a lower average rating

In [27]:
# Aggregate our data by player
# can see that travarious has a high passer rating
# Brandon weeden has a low passer rating

alt.Chart(df).mark_bar().encode(
    x = "qb",
    y = "average(rate)"
)

alt.Chart(...)

## Using the tooltip you can compare various factors of players
## You can identiy Travarious Jackson and Brandon Weeden in the dataset

In [26]:
# Build a SPLOM
alt.Chart(df).mark_circle().encode(
    alt.X(alt.repeat("column"), type="quantitative"),
    alt.Y(alt.repeat("row"), type="quantitative"),
    color="qb",
    tooltip=["qb"]
).properties(
    width=150,
    height=150
).repeat(
    row=["att", "cmp", "td", "rate"],
    column=["att", "cmp", "td", "rate"]
).interactive()

alt.RepeatChart(...)

In [ ]:
# Keep getting error here! IDK =( 

In [29]:
# Predict qb as a function of rate (mirroring regression)
# Note that x should be a dataframe (hence the double [])
x = df[["rate"]]
y = df["qb"]

# Run kNN with one neighbor
runKNN(df, x, y, 1)

# Run kNN with five neighbors
runKNN(df, x, y, 5)

# Measure performance for different values for k
for k in range(1, 10): 
    print(str(k) + " neighbors:")
    runKNN(df, x, y, k)
    
# Print out the number of datapoints in each class
y.value_counts()

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [ ]:
# Getting same error...how can I fox this?

In [44]:
# Now let's run with a broader set of data
# We can use ALL of our numeric data to make a prediction
x = df.select_dtypes(include="number")
y = df["rate"]

# Run k-NN for two different values of k
runKNN(df, x, y, 1)
runKNN(df, x, y, 5)

# Find the best k to predict position. 
# We can do this by looking at the accuracy for each setting of k
# and choosing the k that gives us the best performance
for k in range(1, 10): 
    print(str(k) + " Neighbors:")
    runKNN(df, x, y, k)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [ ]:
# Same error! SADNESS...

In [31]:
# Use the model to analyze player performance
# Let's start with Nick Foleson
player = df[df["qb"] == "Nick FolesN. Foles"]
print(player)
playerData = player.select_dtypes(include="number")

# Build a model to use in prediction. We can see that our classifier doesn't get Ovechkin's position
# right, but is close (a LW and RW play the same general position, just on different halves of the ice)
model = runKNN(df, x, y, 7)
print("k-NN thinks " + player["qb"] + " is a " + model.predict(playerData)[0])

# We can print out the n players closest to our target (control n by changing n_neighbors). 
# Note that the players are listed in order from closest to farthest
relatedPlayers = model.kneighbors(playerData, n_neighbors=3, return_distance=False)

for p in relatedPlayers[0]: 
    print(df.iloc[int(p)]["qb"])

Empty DataFrame
Columns: [qb, att, cmp, yds, ypa, td, int, lg, sack, loss, rate, game_points, home_away, year]
Index: []


ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [ ]:
# This probably won't work since the ttwo above did not! 

In [40]:
# Let's compare the predictions made by k-NN to those made with PCA
# PCA says that Kesler will be most similar to Voracek and Stepan
# The differences have to do with the fact that PCA processes the 
# data to find meaningful mathematical relationships where k-NN looks
# at raw differences in the data. 
player = df[df["qb"] == "Nick FolesN. Foles"]
relatedPlayers = model.kneighbors(player.select_dtypes(include="number"), n_neighbors=5, return_distance=False)

for p in relatedPlayers[0]: 
    print(df.iloc[int(p)]["qb"])

AttributeError: 'KMeans' object has no attribute 'kneighbors'

In [45]:
# We can use k-Means to find general clusters of like players in our data 
#(i.e., without a specific player in mind)

# Run k-Means on the data. Notice that this doesn't show us too much yet
runKMeans(df, 5)

# Choose the best k for k-Means. Let's give it a really big range
# to start with. Notice that I'm using a constant here so we can just
# change this value in one place to test different ranges

MAX_K = 15

scores = []
for k in range(1, MAX_K):
    model = runKMeans(df, k)
    
    # Inertia corresponds to the sum squared distances between points
    # and their centroids/means
    scores.append(model.inertia_)
    
# Let's create a dataframe so we can visualize performance for different ks 
# and choose the best one. In this case, a good k is between 5 and 10: we 
# see consistent performance gains but aren't likely to overfit
scoresDf = pd.DataFrame()
scoresDf["k"] = range(1, MAX_K)
scoresDf["Score"] = scores

alt.Chart(scoresDf).mark_line().encode(
    x = "k",
    y = "Score"
)

alt.Chart(...)

In [46]:
# Let's use the model to explore our data
model = runKMeans(df, 7)

# What are the stats of our prototypical players?
# These correspond to the cluster centers (the means in k-Means)
print(df.columns)
print(model.cluster_centers_)

# (Added after class) The code below will print them out a little 
# more neatly
features = df.select_dtypes(include="number")
for c in range(0, len(model.cluster_centers_)):
    print("***** Cluster " + str(c))
    for f in range(0, len(features.columns)):
        feature = features.columns[f]
        print("Mean " + feature + " = " + str(model.cluster_centers_[c][f]))

# Analyzing text descriptions of these centers is not a lot of fun
# We can instead look at sets of related players. 
# Create a scatterplot matrix of the data with potentially useful stats
# ATOI is a particularly informative feature here. 
alt.Chart(df).mark_circle().encode(
    alt.X(alt.repeat("column"), type="quantitative"),
    alt.Y(alt.repeat("row"), type="quantitative"),
    color="Cluster:N",
    tooltip=["qb"]
).properties(
    width=100,
    height=100
).repeat(
    row=["att", "cmp", "td", "rate"],
    column=["att", "cmp", "td", "rate"]
).interactive()

Index(['qb', 'att', 'cmp', 'yds', 'ypa', 'td', 'int', 'lg', 'sack', 'loss',
       'rate', 'game_points', 'home_away', 'year', 'Cluster'],
      dtype='object')
[[3.54827586e+01 2.06896552e+01 2.26931034e+02 6.49310345e+00
  1.06896552e+00 1.10344828e+00 3.55172414e+00 2.31034483e+01
  7.49931034e+01 1.84137931e+01 2.01300000e+03 6.00000000e+00]
 [2.37500000e+00 1.00000000e+00 1.07500000e+01 4.93750000e+00
  1.25000000e-01 1.25000000e-01 0.00000000e+00 0.00000000e+00
  6.48625000e+01 1.48750000e+01 2.01300000e+03 6.25000000e+00]
 [4.13333333e+01 2.90000000e+01 4.41833333e+02 1.07500000e+01
  3.83333333e+00 5.00000000e-01 2.50000000e+00 1.81666667e+01
  1.27750000e+02 3.58333333e+01 2.01300000e+03 6.66666667e+00]
 [4.45263158e+01 2.80526316e+01 3.40789474e+02 7.84210526e+00
  1.94736842e+00 1.26315789e+00 2.36842105e+00 1.59473684e+01
  9.14157895e+01 2.48947368e+01 2.01300000e+03 6.52631579e+00]
 [2.56923077e+01 1.63846154e+01 1.75923077e+02 7.55384615e+00
  1.84615385e+00 1.53846154e-

alt.RepeatChart(...)

In [ ]:
# Weeden in yellow cluster 5 --> 
# Jackson in green cluster 4 --> higher avg TD high, lower int, lower sack, higher avg rate